In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
import math

In [2]:
df=np.loadtxt('Net1_subset.txt')
df = sparse.csr_matrix(df)  
print(df)



  (0, 5855)	5.0
  (0, 6285)	5.0
  (0, 6359)	5.0
  (0, 7762)	4.0
  (0, 8411)	2.0
  (0, 9083)	3.0
  (1, 36)	5.0
  (1, 807)	4.0
  (1, 1147)	4.0
  (1, 1474)	4.0
  (1, 1682)	1.0
  (1, 2261)	5.0
  (1, 2494)	3.0
  (1, 3553)	3.0
  (1, 3936)	5.0
  (1, 4474)	5.0
  (1, 4998)	4.0
  (1, 5375)	4.0
  (1, 5794)	1.0
  (1, 5976)	5.0
  (1, 6960)	5.0
  (1, 7205)	5.0
  (1, 7891)	4.0
  (1, 9298)	3.0
  (2, 4499)	3.0
  :	:
  (8137, 8491)	4.0
  (8137, 8520)	4.0
  (8137, 8738)	3.0
  (8138, 86)	5.0
  (8138, 1242)	5.0
  (8138, 1369)	5.0
  (8138, 2233)	2.0
  (8138, 2353)	4.0
  (8138, 2718)	4.0
  (8138, 5594)	3.0
  (8138, 7073)	4.0
  (8138, 7438)	5.0
  (8138, 7552)	4.0
  (8139, 5342)	5.0
  (8139, 8553)	5.0
  (8139, 8671)	5.0
  (8139, 8868)	5.0
  (8139, 8979)	3.0
  (8139, 9067)	5.0
  (8140, 3259)	5.0
  (8140, 8140)	3.0
  (8140, 8549)	4.0
  (8140, 8553)	5.0
  (8140, 8868)	5.0
  (8140, 8979)	5.0


In [3]:
print (df.shape,'   ',type(df))

(8141, 9318)     <class 'scipy.sparse.csr.csr_matrix'>


In [4]:
rows_list = []
start = 0
for i, end in enumerate(df.indptr[1:]):
    for j, val in zip(df.indices[start:end], df.data[start:end]):
        x1=[i,j,val]
        rows_list.append(x1)
    start = end
    
ratings = pd.DataFrame(rows_list,columns=['userId','itemId','rating'])
print (ratings.head())

   userId  itemId  rating
0       0    5855     5.0
1       0    6285     5.0
2       0    6359     5.0
3       0    7762     4.0
4       0    8411     2.0


In [5]:
ratings.describe()

,userId,itemId,rating
count,196656.000000,196656.000000,196656.000000
mean,4047.150217,4843.988711,3.498103
std,2374.198911,2757.970038,1.153609
min,0.000000,0.000000,1.000000
25%,1885.000000,2452.000000,3.000000
50%,3971.000000,4883.000000,4.000000
75%,6160.000000,7304.000000,4.000000
max,8140.000000,9317.000000,5.000000


In [6]:
total_user = np.zeros(8141)
count = np.zeros(8141,dtype='int64')

In [7]:
for i,j in ratings.iterrows():
    k=int(j[0])
    count[k]+=1

In [8]:
number_of_test = [math.floor(0.2*i) for i in count]
number_of_holdout = [math.floor(0.1*i) for i in count]
number_of_test = np.array(number_of_test,dtype='int64')
number_of_holdout = np.array(number_of_holdout,dtype='int64')
total_test_holdout = number_of_test + number_of_holdout
number_of_train = [j-i for i,j in zip(total_test_holdout,count)]

In [12]:
train_data = pd.DataFrame()
holdout_data = pd.DataFrame()
test_data = pd.DataFrame()

In [13]:
current_train_size = np.zeros(8141,dtype='int64')
current_holdout_size = np.zeros(8141,dtype='int64')

In [14]:
for i,j in ratings.iterrows():
    user=int(j[0])
    if current_train_size[user] < number_of_train[user]:
        train_data = train_data.append(j)
        current_train_size[user]+= 1
    elif current_holdout_size[user] < number_of_holdout[user]:
        holdout_data=holdout_data.append(j)
        current_holdout_size[user]+= 1
    else:
        test_data = test_data.append(j)
    
###reshuffling data###
test_data = test_data.sample(frac=1).reset_index(drop=True)
train_data = train_data.sample(frac=1).reset_index(drop=True)
holdout_data = holdout_data.sample(frac=1).reset_index(drop=True)

KeyboardInterrupt: 

In [ ]:
train_data.to_csv('train.csv', sep=',')
test_data.to_csv('test.csv',sep=',')
holdout_data.to_csv('holdout.csv',sep=',')

### Movie mean

In [59]:
movies_mean=np.zeros(shape=[9318,3])
user_total = np.zeros(8141)
count = np.zeros(8141,dtype='int64')
for i,j in tr_temp.iterrows():
    k=int(j['userId'])
    count[k]+= 1
    user_total[k]+= j['rating']
    p = int(j['itemId'])
    movies_mean[p][0]+= 1
    movies_mean[p][1]+= j['rating']
    
user_mean=np.array([ i/j for i,j in zip(user_total,count) ])
for x in movies_mean:
    x[2]=round(x[1]/x[0],2)
    
for x in movies_mean:
    if math.isnan(x[2]):
        x[2]=0